In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import pickle
from utils import lla2enu, ecef2enu, get_google_data, find_nearest_time, find_nearest_pos

## GPS and IMU measurements

In [2]:
# read IMU data
imu_path = "../data/csv_data/imu.csv"
imu = pd.read_csv(imu_path)
imu['rosbagTimestamp'] = imu['secs']*int(1e9) + imu['nsecs']
imu = imu.drop(['header', 'seq', 'stamp', 'secs', 'nsecs'], axis=1)

imu.rename(columns={'frame_id':'has_imu', 'rosbagTimestamp':'time'}, inplace=True)
imu['has_imu'] = True

imu.rename(columns={f'{i}': f'imu_{i}' for i in ['ang_vel_x', 'ang_vel_y', 'ang_vel_z', 'lin_acc_x', 'lin_acc_y', 'lin_acc_z']}, inplace=True)
imu

,time,has_imu,imu_ang_vel_x,imu_ang_vel_y,imu_ang_vel_z,imu_lin_acc_x,imu_lin_acc_y,imu_lin_acc_z
0,1508987519143971920,True,0.002271,-0.004726,0.012533,-0.313470,0.121048,-0.281582
1,1508987519147371053,True,0.004968,-0.005145,0.011527,-0.455330,0.147042,-0.183228
2,1508987519153815031,True,0.008092,-0.003727,0.011873,-0.591017,0.290014,0.006007
3,1508987519157360076,True,0.010363,0.000223,0.011455,-0.624363,0.199674,0.009783
4,1508987519163784980,True,0.007671,0.000836,0.012655,-0.525673,0.066757,-0.181699
...,...,...,...,...,...,...,...,...
155236,1508988347454502105,True,0.011993,0.015935,-0.005347,0.502658,-0.468592,0.672039
155237,1508988347461013078,True,-0.000925,0.008811,-0.000713,0.568252,-0.228897,0.605711
155238,1508988347464492082,True,-0.006713,-0.002176,0.003824,0.530415,0.303166,0.301102
155239,1508988347471018075,True,0.006498,-0.001844,0.005197,0.316711,0.663943,0.297036


In [3]:
# read GPS data
gps_path = '../data/csv_data/gps.csv'
gps = pd.read_csv(gps_path)
gps.drop(['header', 'seq', 'stamp', 'secs', 'nsecs', 'status', 'status.1', 'service', 'position_covariance', 'position_covariance_type'], axis=1, inplace=True)
gps.rename(columns={'frame_id':'has_gps', 'rosbagTimestamp':'time'}, inplace=True)
gps['has_gps'] = True

# convert to local ECEF (linearized around ORIGIN_LAT, ORIGIN_LON, ORIGIN_ALT defined in utils.py)
gps[['gps_pos_x', 'gps_pos_y', 'gps_pos_z']] = gps.apply(lambda row: lla2enu(row.latitude, row.longitude, row.altitude), axis=1, result_type='expand')
gps.drop(['latitude', 'longitude', 'altitude'], axis=1, inplace=True)

gps

,time,has_gps,gps_pos_x,gps_pos_y,gps_pos_z
0,1508987519143971920,True,-36.924676,487.124779,0.867260
1,1508987519147371053,True,-36.952543,487.177208,0.867159
2,1508987519153815031,True,-36.980399,487.229636,0.867061
3,1508987519157360076,True,-37.008254,487.282048,0.866966
4,1508987519163784980,True,-37.036110,487.334451,0.866873
...,...,...,...,...,...
155236,1508988347454502105,True,-7960.995047,-9905.376454,2.267097
155237,1508988347461013078,True,-7961.093686,-9905.429942,2.264516
155238,1508988347464492082,True,-7961.192325,-9905.483446,2.261919
155239,1508988347471018075,True,-7961.290975,-9905.536942,2.259314


In [4]:
# merge GPS and IMU
meas = pd.concat([gps, imu]).sort_values('time')
meas = meas.groupby('time').aggregate(max).reset_index()
meas

,time,has_gps,gps_pos_x,gps_pos_y,gps_pos_z,has_imu,imu_ang_vel_x,imu_ang_vel_y,imu_ang_vel_z,imu_lin_acc_x,imu_lin_acc_y,imu_lin_acc_z
0,1508987519143971920,True,-36.924676,487.124779,0.867260,True,0.002271,-0.004726,0.012533,-0.313470,0.121048,-0.281582
1,1508987519147371053,True,-36.952543,487.177208,0.867159,True,0.004968,-0.005145,0.011527,-0.455330,0.147042,-0.183228
2,1508987519153815031,True,-36.980399,487.229636,0.867061,True,0.008092,-0.003727,0.011873,-0.591017,0.290014,0.006007
3,1508987519157360076,True,-37.008254,487.282048,0.866966,True,0.010363,0.000223,0.011455,-0.624363,0.199674,0.009783
4,1508987519163784980,True,-37.036110,487.334451,0.866873,True,0.007671,0.000836,0.012655,-0.525673,0.066757,-0.181699
...,...,...,...,...,...,...,...,...,...,...,...,...
155234,1508988347454502105,True,-7960.995047,-9905.376454,2.267097,True,0.011993,0.015935,-0.005347,0.502658,-0.468592,0.672039
155235,1508988347461013078,True,-7961.093686,-9905.429942,2.264516,True,-0.000925,0.008811,-0.000713,0.568252,-0.228897,0.605711
155236,1508988347464492082,True,-7961.192325,-9905.483446,2.261919,True,-0.006713,-0.002176,0.003824,0.530415,0.303166,0.301102
155237,1508988347471018075,True,-7961.290975,-9905.536942,2.259314,True,0.006498,-0.001844,0.005197,0.316711,0.663943,0.297036


## GPS ground truth

In [5]:
# ground_truth = pd.DataFrame(columns={'time', 'x', 'y', 'z'})
# ground_truth.to_csv('../data/ground_truth.csv')

## Google maps

In [6]:
google = get_google_data('../data/map_to_gpx.gpx', alt=np.zeros(500000)).T
road = pd.DataFrame({'road_lat':google[0], 'road_lon':google[1], 'road_alt':google[2]})
road[['road_x', 'road_y', 'road_z']] = road.apply(lambda row: ecef2enu(row.road_lat, row.road_lon, row.road_alt), axis=1, result_type='expand')
road.drop(['road_lat', 'road_lon', 'road_alt'], axis=1, inplace=True)
road

,road_x,road_y,road_z
0,-30.985232,487.403826,-146.018670
1,-31.000301,487.433635,-146.018673
2,-31.015371,487.463444,-146.018675
3,-31.030440,487.493253,-146.018677
4,-31.045510,487.523062,-146.018680
...,...,...,...
499995,-7961.829607,-9908.883879,-158.665599
499996,-7961.845342,-9908.914852,-158.665667
499997,-7961.861203,-9908.945516,-158.665734
499998,-7961.877275,-9908.975678,-158.665801


## Camera data

In [9]:
with open("../data/deviation_from_center", "rb") as fp:
    dev_from_center = pickle.load(fp)
    dev_from_center = pd.DataFrame({'delta':dev_from_center})
    dev_from_center = pd.DataFrame({'delta':dev_from_center.apply(lambda row: row.delta * (3.05/3.5), axis=1)}) # BUG: Quick fix for wrong LW in utils.py

with open("../data/img_data_dir", "rb") as fp:
    cam_timestamps = pickle.load(fp)
    cam_timestamps = [int(cam_timestamp.split('.')[0])*1000 for cam_timestamp in cam_timestamps]
    cam_timestamps = pd.DataFrame({'cam_timestamps':cam_timestamps})

cam_times = find_nearest_time(gps['time'], cam_timestamps['cam_timestamps'])
cam_times['has_cam'] = True
cam = cam_times.join(dev_from_center).sort_values('time')
cam

,time,has_cam,delta
0,1508987519233833074,True,0.210874
1,1508987519403791904,True,0.228041
2,1508987519573848962,True,0.239848
3,1508987519743845939,True,0.187404
4,1508987519913876056,True,0.216284
...,...,...,...
4872,1508988346560975074,True,0.184641
4873,1508988346730956077,True,0.178853
4874,1508988346900969028,True,0.178133
4875,1508988347064472913,True,0.152686


In [10]:
cam_pos = gps.merge(cam, on='time')[['time', 'gps_pos_x', 'gps_pos_y']]
centerlane = find_nearest_pos(road['road_x'], road['road_y'], cam_pos['gps_pos_x'], cam_pos['gps_pos_y'])
cam_pos = cam_pos.join(centerlane).drop(['gps_pos_x', 'gps_pos_y'], axis=1)
cam = cam.merge(cam_pos, on='time')
cam

[4870/4877]

,time,has_cam,delta,road_x,road_y
0,1508987519233833074,True,0.210874,-32.568157,490.533462
1,1508987519403791904,True,0.228041,-33.473377,492.321482
2,1508987519573848962,True,0.239848,-34.379206,494.109191
3,1508987519743845939,True,0.187404,-35.270616,495.866755
4,1508987519913876056,True,0.216284,-36.162779,497.623927
...,...,...,...,...,...
4872,1508988346560975074,True,0.184641,-7943.567638,-9895.763345
4873,1508988346730956077,True,0.178853,-7946.906902,-9897.479035
4874,1508988346900969028,True,0.178133,-7950.189840,-9899.341506
4875,1508988347064472913,True,0.152686,-7953.272083,-9901.307420


In [11]:
# Add current lane, LANEWIDTH, number of lanes

# Taken from images
idxs  = np.array([0, 242, 250, 761, 794, 838, 872, 1098, 1130, 1724, 1751, 2215, 2291, 2943, 3509, 3607, 4877]) # frame index where a lane change happens
lanes = np.array([     1, 0.5,   0,   0,   1,   0,    0,    1,    0,  0.5,    1,    0,    1,    0,    1,    2]) # lane the car is in, rightmost=0
lens  = np.diff(idxs)
curr_lane = np.empty(0)
for le, lane in zip(lens, lanes):
    curr_lane = np.hstack((curr_lane, np.ones(le)*lane))
cam['curr_lane'] = curr_lane

cam['LANEWIDTH'] = 3.05

cam['n_lanes']   = 3
cam

,time,has_cam,delta,road_x,road_y,curr_lane,LANEWIDTH,n_lanes
0,1508987519233833074,True,0.210874,-32.568157,490.533462,1.0,3.05,3
1,1508987519403791904,True,0.228041,-33.473377,492.321482,1.0,3.05,3
2,1508987519573848962,True,0.239848,-34.379206,494.109191,1.0,3.05,3
3,1508987519743845939,True,0.187404,-35.270616,495.866755,1.0,3.05,3
4,1508987519913876056,True,0.216284,-36.162779,497.623927,1.0,3.05,3
...,...,...,...,...,...,...,...,...
4872,1508988346560975074,True,0.184641,-7943.567638,-9895.763345,2.0,3.05,3
4873,1508988346730956077,True,0.178853,-7946.906902,-9897.479035,2.0,3.05,3
4874,1508988346900969028,True,0.178133,-7950.189840,-9899.341506,2.0,3.05,3
4875,1508988347064472913,True,0.152686,-7953.272083,-9901.307420,2.0,3.05,3


In [12]:
# Add camera data to meas
meas = meas.join(cam.set_index('time'), on='time')
meas = meas.fillna(value={"has_cam": False})
meas['has_cam'] = meas.apply(lambda row: False if np.isnan(row.delta) else True, axis=1)
meas

,time,has_gps,gps_pos_x,gps_pos_y,gps_pos_z,has_imu,imu_ang_vel_x,imu_ang_vel_y,imu_ang_vel_z,imu_lin_acc_x,imu_lin_acc_y,imu_lin_acc_z,has_cam,delta,road_x,road_y,curr_lane,LANEWIDTH,n_lanes
0,1508987519143971920,True,-36.924676,487.124779,0.867260,True,0.002271,-0.004726,0.012533,-0.313470,0.121048,-0.281582,False,NaN,NaN,NaN,NaN,NaN,NaN
1,1508987519147371053,True,-36.952543,487.177208,0.867159,True,0.004968,-0.005145,0.011527,-0.455330,0.147042,-0.183228,False,NaN,NaN,NaN,NaN,NaN,NaN
2,1508987519153815031,True,-36.980399,487.229636,0.867061,True,0.008092,-0.003727,0.011873,-0.591017,0.290014,0.006007,False,NaN,NaN,NaN,NaN,NaN,NaN
3,1508987519157360076,True,-37.008254,487.282048,0.866966,True,0.010363,0.000223,0.011455,-0.624363,0.199674,0.009783,False,NaN,NaN,NaN,NaN,NaN,NaN
4,1508987519163784980,True,-37.036110,487.334451,0.866873,True,0.007671,0.000836,0.012655,-0.525673,0.066757,-0.181699,False,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155234,1508988347454502105,True,-7960.995047,-9905.376454,2.267097,True,0.011993,0.015935,-0.005347,0.502658,-0.468592,0.672039,False,NaN,NaN,NaN,NaN,NaN,NaN
155235,1508988347461013078,True,-7961.093686,-9905.429942,2.264516,True,-0.000925,0.008811,-0.000713,0.568252,-0.228897,0.605711,False,NaN,NaN,NaN,NaN,NaN,NaN
155236,1508988347464492082,True,-7961.192325,-9905.483446,2.261919,True,-0.006713,-0.002176,0.003824,0.530415,0.303166,0.301102,False,NaN,NaN,NaN,NaN,NaN,NaN
155237,1508988347471018075,True,-7961.290975,-9905.536942,2.259314,True,0.006498,-0.001844,0.005197,0.316711,0.663943,0.297036,False,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
meas.to_csv('../data/measurements.csv')